In [6]:
%pip install llama-index-postprocessor-cohere-rerank

     ---------------------------------------- 52.7/52.7 KB 2.8 MB/s eta 0:00:00
  Using cached importlib_metadata-6.11.0-py3-none-any.whl (23 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
     -------------------------------------- 546.3/546.3 KB 8.6 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib_metadata 7.0.2
    Uninstalling importlib_metadata-7.0.2:
      Successfully uninstalled importlib_metadata-7.0.2
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\llamaindex\llamaindex-advanced\llama-10\Scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
!pip install llama-index python-dotenv

You should consider upgrading via the 'C:\llamaindex\llamaindex-advanced\llama-10\Scripts\python.exe -m pip install --upgrade pip' command.


In [8]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader
)

from llama_index.core.response.pprint_utils import pprint_response

import os
from dotenv import load_dotenv 

load_dotenv(".env", override=True)
openai_key = os.environ["OPENAI_API_KEY"] 

In [9]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()
# build index
index = VectorStoreIndex.from_documents(documents=documents)



In [11]:
import os
from llama_index.postprocessor.cohere_rerank import CohereRerank

api_key = os.environ["COHERE_API_KEY"]
cohere_rerank = CohereRerank(api_key=api_key, top_n=2)

In [12]:
query_engine = index.as_query_engine(
    similarity_top_k=2,
)
response = query_engine.query(
    "What did sam altman do in this essay?",
)

In [13]:
# pprint.pprint(response)
pprint_response(response,show_source=True)

Final Response: Sam Altman was asked to become the president of Y
Combinator (YC) in the essay.
______________________________________________________________________
Source Node 1/2
Node ID: 2c922cbd-5624-4e1c-bc52-d8832e273011
Similarity: 0.8188785404081247
Text: This seemed strange advice, because YC was doing great. But if
there was one thing rarer than Rtm offering advice, it was Rtm being
wrong. So this set me thinking. It was true that on my current
trajectory, YC would be the last thing I did, because it was only
taking up more of my attention. It had already eaten Arc, and was in
the process of ea...
______________________________________________________________________
Source Node 2/2
Node ID: 6bbaab15-e1cf-4acd-9d7f-a7098b4fcd51
Similarity: 0.8067207471928
Text: Socially they'd seem more like rants posted by nutjobs on their
GeoCities sites than the genteel and beautifully typeset compositions
published in The New Yorker. But by this point I knew enough to find
that encourag

In [14]:

query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank],
)
response = query_engine.query(
    "What did Sam Altman do in this essay?",
)

In [15]:
pprint_response(response,show_source=True)

Final Response: Sam Altman was one of the individuals who participated
in the Summer Founders Program mentioned in the essay.
______________________________________________________________________
Source Node 1/2
Node ID: 676709b7-40a2-4646-9b16-6b81d0e3ea1d
Similarity: 0.81286734
Text: Why not organize a summer program where they'd start startups
instead? We wouldn't feel guilty for being in a sense fake investors,
because they would in a similar sense be fake founders. So while we
probably wouldn't make much money out of it, we'd at least get to
practice being investors on them, and they for their part would
probably have a m...
______________________________________________________________________
Source Node 2/2
Node ID: 92b6e210-3d88-4329-8a84-20defec177c7
Similarity: 0.70007527
Text: But after Heroku got bought we had enough money to go back to
being self-funded.    [15] I've never liked the term "deal flow,"
because it implies that the number of new startups at any given time
is

In [16]:
from llama_index.core import QueryBundle
from llama_index.core.schema import TextNode, NodeWithScore

query = QueryBundle("I want a dog.")

scored_nodes = [
    NodeWithScore(node=TextNode(text="This is a cat"), score=0.6),
    NodeWithScore(node=TextNode(text="This is a dog"), score=0.4),
]

In [17]:
reranked_nodes = cohere_rerank.postprocess_nodes(
    scored_nodes, query_bundle=query
)

In [18]:

print([(x.text, x.score) for x in reranked_nodes])

[('This is a dog', 0.011597361), ('This is a cat', 0.0073458264)]
